# Programming Assignment 2

Here is the code snippet for the dqn algorithm.
```python
def dqn(agent, env, n_episodes=100, max_t=500):

    scores_window = deque(maxlen=100)
    ''' last 100 scores for checking if the avg is more than 195 '''

    scores_over_episodes=[]

    ''' initialize epsilon '''

    for i_episode in range(1, n_episodes+1):
        state = env.reset()[0]
        score = 0
        for t in range(max_t):
            action = agent.act(state)
            next_state, reward, done, _, _ = env.step(action)
            agent.step(state, action, reward, next_state, done)
            state = next_state
            score += reward
            if done:
                break

        scores_window.append(score)
        scores_over_episodes.append(score)

        ''' decrease epsilon '''

        print('\rEpisode {}\tAverage Score: {:.2f}\tNumber of steps: {}'.format(i_episode, np.mean(scores_window), t), end="")

        if i_episode % 100 == 0:
           print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)))
        """if np.mean(scores_window)>=195.0:
           print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)))
           break"""


    return scores_over_episodes
```

### Imports

In [5]:
import gym
from duelingDQN import dqn, DuelingDQNAgent
import matplotlib.pyplot as plt
from vars import *
import numpy as np

ENVS = [HyperEnv('CartPole-v1', 150, 1000, 128, 1e-4, 100, 0.99, 0.1, 128, 64), HyperEnv('Acrobot-v1', 200, 500, 256, 1e-3, 500, 0.99, 0.1, 128, 256)]

## DDQN Cartpole

### Dueling DQN algorithm with mean aggregation

In [ ]:
cartpole_parameters = ENVS[0]

print(f"\nTraining and evaluating Dueling DQN agent on {cartpole_parameters.name} environment:")
env = gym.make(cartpole_parameters.name)

rewards = []
for _ in range(5):
    dueling_dqn_agent = DuelingDQNAgent(env.observation_space.shape[0], env.action_space.n, 37, "mean", cartpole_parameters)
    scores = dqn(dueling_dqn_agent, env, n_episodes=cartpole_parameters.n_episodes, max_t=cartpole_parameters.max_t)
    rewards.append(scores)
avg_reward = np.array(rewards).mean(0)
std_reward = np.array(rewards).std(0)

x_axis = np.arange(len(rewards[0]))
plt.fill_between(x_axis, avg_reward - std_reward, avg_reward + std_reward, alpha=0.5)
plt.title("Reward vs Episode")
plt.plot(avg_reward)
plt.show()


Training and evaluating Dueling DQN agent on CartPole-v1 environment:
Episode 100	Score: 178.00
Episode 100	Score: 153.00
Episode 131	Score: 216.00

### Dueling DQN algorithm with max aggregation

In [ ]:
cartpole_parameters = ENVS[0]
seed = 37

print(f"\nTraining and evaluating Dueling DQN agent on {cartpole_parameters.name} environment:")
env = gym.make(cartpole_parameters.name)

rewards = []
for _ in range(5):
    dueling_dqn_agent = DuelingDQNAgent(env.observation_space.shape[0], env.action_space.n, seed, "max", cartpole_parameters)
    scores = dqn(dueling_dqn_agent, env, n_episodes=cartpole_parameters.n_episodes, max_t=cartpole_parameters.max_t)
    rewards.append(scores)
avg_reward = np.array(rewards).mean(0)
std_reward = np.array(rewards).std(0)

x_axis = np.arange(len(rewards[0]))
plt.fill_between(x_axis, avg_reward - std_reward, avg_reward + std_reward, alpha=0.5)
plt.plot(avg_reward)
plt.title("Reward vs Episode")
plt.show()

## DDQN Acrobot

### Dueling DQN algorithm with mean aggregation on Acrobot environment

In [ ]:
acrobot_parameters = ENVS[1]

print(f"\nTraining and evaluating Dueling DQN agent on {ENVS[1].name} environment:")
env = gym.make(acrobot_parameters.name)
rewards = []
for _ in range(5):
    dueling_dqn_agent = DuelingDQNAgent(env.observation_space.shape[0], env.action_space.n, 37, "mean", acrobot_parameters)
    scores = dqn(dueling_dqn_agent, env, n_episodes=acrobot_parameters.n_episodes, max_t=acrobot_parameters.max_t)
    rewards.append(scores)
avg_reward = np.array(rewards).mean(0)
std_reward = np.array(rewards).std(0)

x_axis = np.arange(len(rewards[0]))
plt.fill_between(x_axis, avg_reward - std_reward, avg_reward + std_reward, alpha=0.5)
plt.plot(avg_reward)
plt.title("Reward vs Episode")
plt.show()

### Dueling DQN algorithm with max aggregation on Acrobot environment

In [ ]:
acrobot_parameters = ENVS[1]

print(f"\nTraining and evaluating Dueling DQN agent on {ENVS[1].name} environment:")
env = gym.make(acrobot_parameters.name)
rewards = []
for _ in range(5):
    dueling_dqn_agent = DuelingDQNAgent(env.observation_space.shape[0], env.action_space.n, 37, "max", acrobot_parameters)
    scores = dqn(dueling_dqn_agent, env, n_episodes=acrobot_parameters.n_episodes, max_t=acrobot_parameters.max_t)
    rewards.append(scores)
avg_reward = np.array(rewards).mean(0)
std_reward = np.array(rewards).std(0)

x_axis = np.arange(len(rewards[0]))
plt.fill_between(x_axis, avg_reward - std_reward, avg_reward + std_reward, alpha=0.5)
plt.plot(avg_reward)
plt.title("Reward vs Episode")
plt.show()

## DDQN Inferences

## REINFORCE cartpole